<h1 align="center">Devoir #3</h1>
<h4 align="center">Théo Dionne et Jérôme Leblanc</h4>
<h5 align="center">F I I I I L L L L    O U T T T</h5>

## To-do
- Faire les commentaires au complet
- Peut-être restructurer le calisse de spaghetti

In [ ]:
import numpy as np
from scipy.optimize import brentq
from scipy.integrate import odeint
import OHQ_wrapper as OHQw

import matplotlib.pyplot as plt

params = {
    "xtick.top": True, 
    "ytick.right": True, 
    "xtick.direction": "in", 
    "ytick.direction": "in", 
    "figure.dpi": 200
    }
plt.rcParams.update(params)

from solvers import solutions_mt, solutions_mef

# Méthode du tir

D'abord, on cherche à résoudre l'équation de Schrödinger avec la méthode du tir. Le hamiltonien est de la forme d'un oscillateur harmonique, soit:
$$
    {\psi}'' = (x^2-2E)\psi
$$
Comme à l'habitude, pour résoudre cette équation du second ordre, on s'intéresse à un système *réduit* avec le changement de variables typique $\xi \equiv \psi'$:
$$
    \begin{bmatrix}{\psi}'\\{\xi}'\end{bmatrix} = \begin{bmatrix}\xi\\(x^2-2E)\psi\end{bmatrix}
$$
Maintenant, on fixe $\psi(-L)=0$ et ${\psi}'(-L) = 0.001$ pour ensuite trouver les valeurs de $E$ tel que $\psi(L|E)=0$ 

In [ ]:
sol_mt = OHQw.solutions_mt(5, 200, 6, [0,0.001])

OHQw.faire_graphique(sol_mt, 5, 5)

# Méthode des éléments finis

La méthode des éléments finis consiste essentiellement à ramener une équation différentielle à un système d'équations linéaires diagonalisable en supposant que les fonctions propres sont approximativement leur projection sur la base des *fonctions tente*. Pour ce faire, on ramène l'équation différentielle sous la forme :
$$
    \frac{1}{2}(x^2 - \partial_x^2)\psi = E\psi
$$
En introduisant sa représentation dans la base des fonctions tente $\left\{\ket{u_i}\right\}$, on peut écrire l'équation sous la forme:
$$
    \mathrm{\mathbf{L}}^C\ket{\psi}=E\mathrm{\mathbf{M}}\ket{\psi}
$$
où $\mathrm{\mathbf{L}}^C$ est la représentation de $(x^2 - \partial_x^2)/2$ et $\mathrm{\mathbf{M}}$ est la *matrice de masse*. Les énergies (valeurs propres) et les états (vecteurs propres) sont trouvés via la résolution de cette équation aux valeurs propres généralisées.

In [ ]:
sol_mef = OHQw.solutions_mef(6, 500, 6)

OHQw.faire_graphique(sol_mef, 5, 5)

## Conclusion

AHAHHAHAHAHHAA ÇA FAIT LA MEME CHOSE LMAO

MEF: SOLUTIONS GARANTIES, requiert plus de points et de temps

Tir: pretty good, mais solutions plus dures à cadrer